# Demo for SPEL


In [1]:
%tb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Built-in modules
import re        # Regular expressions
import importlib # MUST BE USED TO RELOAD MODULES
import sys 

# Importing modules for SPEL functions 
import utilityFunctions as uf
import edit_files as ef 
import fortran_modules as fm 
from analyze_subroutines import Subroutine

# mod_config : system configuration and static variables. 
# Where E3SM is stored, where unit tests are stored, etc.
from mod_config import default_mods, unittests_dir, scripts_dir, spel_mods_dir
from mod_config import ELM_SRC, spel_output_dir, _bc

No traceback available to show.


SPEL needs a "casename" (what your unit-test is called) and a list of subroutines to create a unit-test for "sub_name_list" (examples are "LakeTemperature", "SoilTemperature", etc...)


In [2]:
# Define Unit Test parameters 
import os 
casename = "notebook_test"  # Name of the test case
case_dir = unittests_dir + casename # Directory to store the test case

# create directory for unit-test if it does not exist
if(not os.path.isdir(f"{case_dir}") ):
    print(f"Making case directory {case_dir}")
    if(not os.path.isdir(f"{unittests_dir}")):
        os.system(f"mkdir {unittests_dir}")
    os.system(f"mkdir {case_dir}")
    preprocess = True

# List of subroutines to be analyzed
sub_name_list = ["SoilTemperature","LakeTemperature"] 
sub_name_list = [sub.lower() for sub in sub_name_list]

# variables needed for Unit Test
main_sub_dict = {}  # Dictionary to store all Subroutines in files needed for Unit Test
mod_dict = {} 
subroutines = {k.lower():[] for k in sub_name_list} # Dictionary for User Specified Subroutines

1st step for SPEL is determine which modules are needed for LakeTemperature and edit out the I/O, MPI, and other unneccessary modules. This must be somewhat tailored to ELM.

Currently if I module is not present in the "components/elm/src/" (ELM_SRC) nor in the "share/utils" directories (SHR_SRC), then the module and any dependency on it is automatically removed.

In [3]:
 # List to hold all the modules needed for the unit test
needed_mods = [] 
for s in sub_name_list:
    # Get general info of the subroutine
    subroutines[s] = Subroutine(s,calltree=['elm_drv'])

    # Process by removing certain modules and syntax
    # so that a standalone unit test can be compiled.
    fn = subroutines[s].filepath
    mod_dict, file_list, main_sub_dict = ef.process_for_unit_test(
                fname=fn,
                case_dir=case_dir,
                mod_dict=mod_dict,
                mods=needed_mods,
                required_mods=default_mods,
                main_sub_dict=main_sub_dict,
                overwrite=True,
                verbose=False,
            )

    subroutines[s] = main_sub_dict[s]

Adding clumps to clump_type instances
Adding filter to clumpfilter instances
Adding filter_inactive_and_active to clumpfilter instances
dict_keys(['soiltemperaturemod', 'shr_kind_mod', 'decompmod', 'domainmod', 'elm_varctl', 'elm_varcon', 'shr_const_mod', 'urbanparamstype', 'gridcelltype', 'topounit_varcon', 'elm_varsur', 'atm2lndtype', 'canopystatetype', 'solarabsorbedtype', 'soilstatetype', 'ch4varcon', 'pftvarcon', 'sharedparamsmod', 'funcpedotransfermod', 'rootbiophysmod', 'domainlateralmod', 'energyfluxtype', 'annualfluxdribbler', 'topounitdatatype', 'topounittype', 'landunittype', 'landunitdatatype', 'columntype', 'columndatatype', 'soilorder_varcon', 'cnstatetype', 'cndecompcascadecontype', 'vegetationtype', 'vegetationdatatype', 'subgridavemod', 'speciesmod', 'vegetationpropertiestype', 'dynsubgridcontrolmod', 'timeinfomod', 'perfmod_gpu', 'elm_varpar', 'landunit_varcon', 'column_varcon', 'banddiagonalmod', 'subgridmod', 'filtermod'])
dict_keys(['soiltemperaturemod', 'shr_kind_

In [4]:
# Create dictionary containing all identified user defined types
# 'type-name' : 'DerivedType Object'
type_dict = {}
for modname, mod in mod_dict.items():
    for utype, dtype in mod.defined_types.items():
        type_dict[utype] = dtype

SPEL can print out a module tree showing how the modules are linked for the unit-test subroutines
- The full tree can be pretty difficult to parse, having a cutoff depth is recommended.
- Every module uses 'shr_kind_mod' so that could be suppressed as well.

In [5]:
modtree = fm.print_spel_module_dependencies(mod_dict=mod_dict,subs=subroutines)

# Print the module tree
arrow = "-->"
cutoff_depth = 10 # Only print modules up to this depth
suppress_mod_list = ['shr_kind_mod']
for m in modtree:
    depth = m['depth']
    modname = m['module']
    if(modname in suppress_mod_list):
        continue
    if(depth == 1):
        print(_bc.HEADER + arrow*depth + modname + _bc.ENDC)
    elif(depth <= cutoff_depth):
        print( arrow*depth + modname)

soiltemperaturemod
-->decompmod
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->domainmod
-->-->-->elm_varctl
-->elm_varctl
-->elm_varcon
-->-->shr_const_mod
-->-->elm_varpar
-->-->-->elm_varctl
-->-->elm_varctl
-->urbanparamstype
-->-->decompmod
-->-->-->elm_varctl
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->domainmod
-->-->-->-->elm_varctl
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->landunittype
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->gridcelltype
-->-->-->landunit_varcon
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->topounit_varcon
-->-->-->-->elm_varctl
-->-->-->-->elm_varcon
-->--

In [6]:
test_mod = mod_dict['soiltemperaturemod']
test_mod.print_module_info()

Module Name: soiltemperaturemod biogeophys/SoilTemperatureMod.F90
Module Depedencies:
use shr_kind_mod-> shr_kind_r8,
use decompmod-> bounds_type,
use elm_varctl-> iulog, use_petsc_thermal_model,
use elm_varcon-> spval, cnfac, cpice, cpliq, denh2o, denice, tfrz, tkwat, tkice, tkair, thk_bedrock, hfus, grav, sb, hvap, capr,
use urbanparamstype-> urbanparams_type,
use atm2lndtype-> atm2lnd_type,
use canopystatetype-> canopystate_type,
use solarabsorbedtype-> solarabs_type,
use soilstatetype-> soilstate_type,
use energyfluxtype-> energyflux_type,
use topounitdatatype-> top_af,
use landunittype-> lun_pp,
use landunitdatatype-> lun_es, lun_ef,
use columntype-> col_pp,
use columndatatype-> col_es, col_ef, col_ws, col_wf,
use vegetationtype-> veg_pp,
use vegetationdatatype-> veg_ef, veg_wf,
use timeinfomod-> all
use perfmod_gpu-> all
use elm_varpar-> nlevsno, nlevgrnd, nlevurb, nlevsoi, max_patch_per_col,
use landunit_varcon-> istice, istice_mec, istsoil, istcrop, istwet,
use column_varcon-> 

In [7]:
for s in sub_name_list:
    # Parsing means getting info on the variables read and written
    # to by the subroutine and any of its callees
    subroutines[s].parse_subroutine(dtype_dict=type_dict,
                                    main_sub_dict=main_sub_dict,verbose=True)

parse_args:: Inactive args in setrhsvec_snowurbannonroad
['bounds', 'hs_top_snow']
parse_args:: Inactive args in setrhsvec_snowurbanroad
['bounds', 'hs_top', 't_h2osfc']
parse_args:: Inactive args in setrhsvec_snowurban
['bounds', 't_h2osfc']
parse_args:: Inactive args in setrhsvec_snownonurban
['bounds', 'hs_top']
parse_args:: Inactive args in setrhsvec_snow
['bounds']
parse_args::::Adjusted Argumnet line number for setrhsvec_standingsurfacewater in CPP file
parse_args:: Inactive args in setrhsvec_soilurbannonroad
['bounds', 'hs_top_snow', 'hs_soil', 'fn_h2osfc', 'c_h2osfc']
parse_args:: Inactive args in setrhsvec_soilurbanroad
['bounds', 'hs_top', 'fn_h2osfc', 'c_h2osfc']
parse_args:: Inactive args in setrhsvec_soilurban
['bounds', 'fn_h2osfc', 'c_h2osfc']
parse_args:: Inactive args in setrhsvec_soilnonurban
['bounds', 'hs_top', 'fn_h2osfc', 'c_h2osfc']
parse_args::::Adjusted Argumnet line number for setrhsvec_soil_standingsurfacewater in CPP file
parse_args:: Inactive args in setrhs

In [8]:
read_types  = [] 
write_types = []
func_name = "Notebook"
for s in sub_name_list:
    subroutines[s].child_subroutines_analysis(dtype_dict=type_dict,
                                        main_sub_dict=main_sub_dict,verbose=False)

    print(_bc.OKGREEN + f"Derived Type Analysis for {subroutines[s].name}")
    print(f"{func_name}::Read-Only")
    for key in subroutines[s].elmtype_r.keys():
        print(key, subroutines[s].elmtype_r[key])
    print(f"{func_name}::Write-Only")
    for key in subroutines[s].elmtype_w.keys():
        print(key, subroutines[s].elmtype_w[key])
    print(f"{func_name}::Read-Write")
    for key in subroutines[s].elmtype_rw.keys():
        print(key, subroutines[s].elmtype_rw[key])
    print(_bc.ENDC)

    for key in list(subroutines[s].elmtype_r.keys()):
        c13c14 = bool("c13" in key or "c14" in key)
        if c13c14:
            del subroutines[s].elmtype_r[key]
            continue
        if "_inst" in key:
            print(f"error: {key} has _inst")
            sys.exit(1)
        read_types.append(key)

    for key in list(subroutines[s].elmtype_w.keys()):
        c13c14 = bool("c13" in key or "c14" in key)
        if c13c14:
            del subroutines[s].elmtype_w[key]
            continue
        if "_inst" in key:
            print(f"error: {key} has _inst")
            sys.exit(1)
        write_types.append(key)

    for key in list(subroutines[s].elmtype_rw.keys()):
        c13c14 = bool("c13" in key or "c14" in key)
        if c13c14:
            del subroutines[s].elmtype_rw[key]
            continue
        write_types.append(key)

Could be library function?
Creating Subroutine dgbsv in lib.F90 L-999--999
child_subroutines_analysis::dgbsv is a library function -- Skipping.
Derived Type Analysis for soiltemperature
Notebook::Read-Only
lun_pp%urbpoi r
urbanparams_vars%t_building_max r
urbanparams_vars%t_building_min r
col_pp%snl r
col_pp%itype r
col_pp%landunit r
bounds%begl r
bounds%endl r
bounds%begc r
bounds%endc r
col_pp%z r
col_pp%zi r
col_es%t_ssbef r
lun_pp%itype r
col_pp%nlevbed r
col_pp%dz r
urbanparams_vars%tk_wall r
urbanparams_vars%tk_roof r
urbanparams_vars%nlev_improad r
urbanparams_vars%tk_improad r
soilstate_vars%watsat_col r
soilstate_vars%tkmg_col r
soilstate_vars%tkdry_col r
urbanparams_vars%cv_wall r
urbanparams_vars%cv_roof r
urbanparams_vars%cv_improad r
soilstate_vars%csol_col r
col_es%emg r
col_pp%npfts r
col_pp%pfti r
veg_pp%landunit r
veg_pp%topounit r
veg_pp%gridcell r
veg_pp%active r
veg_ef%dlrad r
top_af%lwrad r
veg_ef%eflx_sh_grnd r
veg_wf%qflx_evap_soi r
col_ef%htvp r
veg_ef%eflx_sh_s

In [9]:
# Make sure physical properties types are read/written:
list_pp = ['veg_pp','lun_pp','col_pp','grc_pp','top_pp']

print("read_types:",read_types)
print("write_types:",write_types)

aggregated_elmtypes_list = []
for x in read_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)    
for x in write_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)

# for l in list_pp:
#     aggregated_elmtypes_list.append(l)
print("list of global vars:",aggregated_elmtypes_list)

read_types: ['lun_pp%urbpoi', 'urbanparams_vars%t_building_max', 'urbanparams_vars%t_building_min', 'col_pp%snl', 'col_pp%itype', 'col_pp%landunit', 'bounds%begl', 'bounds%endl', 'bounds%begc', 'bounds%endc', 'col_pp%z', 'col_pp%zi', 'col_es%t_ssbef', 'lun_pp%itype', 'col_pp%nlevbed', 'col_pp%dz', 'urbanparams_vars%tk_wall', 'urbanparams_vars%tk_roof', 'urbanparams_vars%nlev_improad', 'urbanparams_vars%tk_improad', 'soilstate_vars%watsat_col', 'soilstate_vars%tkmg_col', 'soilstate_vars%tkdry_col', 'urbanparams_vars%cv_wall', 'urbanparams_vars%cv_roof', 'urbanparams_vars%cv_improad', 'soilstate_vars%csol_col', 'col_es%emg', 'col_pp%npfts', 'col_pp%pfti', 'veg_pp%landunit', 'veg_pp%topounit', 'veg_pp%gridcell', 'veg_pp%active', 'veg_ef%dlrad', 'top_af%lwrad', 'veg_ef%eflx_sh_grnd', 'veg_wf%qflx_evap_soi', 'col_ef%htvp', 'veg_ef%eflx_sh_snow', 'veg_wf%qflx_ev_snow', 'veg_ef%eflx_sh_soil', 'veg_wf%qflx_ev_soil', 'veg_ef%eflx_sh_h2osfc', 'veg_wf%qflx_ev_h2osfc', 'lun_pp%wtlunit_roof', 'lun_

In [10]:
from UnitTestforELM import set_active_variables
instance_to_user_type = {}
elm_inst_vars = {}
for type_name, dtype in type_dict.items():
    if('bounds' in type_name): 
        continue
    if(not dtype.instances):
        print(f"Warning: no instances found for {type_name}")
        cmd = f'grep -rin -E "^[[:space:]]*(type)[[:space:]]*\({type_name}" {ELM_SRC}/main/elm_instMod.F90'
        output = sp.getoutput(cmd)
        print(f"output: {output}")
        if(output):
            output = output.split('\n')
            if(len(output) > 1):
                print(f"Warning: multiple instances found for {type_name}")
                print(output)
                sys.exit(1)
            line = output[0]
            line = line.replace('::','')
            line = line.split(':')
            
            decl = line[1].strip()
            decl = decl.split()
            var = decl[1]
            new_inst = Variable(type_name,var,subgrid='?',ln=0,dim=0,declaration='elm_instMod')
            dtype.instances.append(new_inst)
            elm_inst_vars[var] = dtype
        else:
            print(f"Warning: no instances found for {type_name}")
    for instance in dtype.instances:
        instance_to_user_type[instance.name] = type_name

dtype_info_list = []
    
for s in sub_name_list:
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_r,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_w,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_rw,dtype_info_list)
    
print(dtype_info_list)

[['lun_pp', 'urbpoi', 'logical', '1D'], ['urbanparams_vars', 't_building_max', 'real', '1D'], ['urbanparams_vars', 't_building_min', 'real', '1D'], ['col_pp', 'snl', 'integer', '1D'], ['col_pp', 'itype', 'integer', '1D'], ['col_pp', 'landunit', 'integer', '1D'], ['col_pp', 'z', 'real', '2D'], ['col_pp', 'zi', 'real', '2D'], ['col_es', 't_ssbef', 'real', '2D'], ['lun_pp', 'itype', 'integer', '1D'], ['col_pp', 'nlevbed', 'integer', '1D'], ['col_pp', 'dz', 'real', '2D'], ['urbanparams_vars', 'tk_wall', 'real', '2D'], ['urbanparams_vars', 'tk_roof', 'real', '2D'], ['urbanparams_vars', 'nlev_improad', 'integer', '1D'], ['urbanparams_vars', 'tk_improad', 'real', '2D'], ['soilstate_vars', 'watsat_col', 'real', '2D'], ['soilstate_vars', 'tkmg_col', 'real', '2D'], ['soilstate_vars', 'tkdry_col', 'real', '2D'], ['urbanparams_vars', 'cv_wall', 'real', '2D'], ['urbanparams_vars', 'cv_roof', 'real', '2D'], ['urbanparams_vars', 'cv_improad', 'real', '2D'], ['soilstate_vars', 'csol_col', 'real', '2D'

In [11]:
for sub in subroutines.values():
    tree = sub.calltree[2:]
    sub.analyze_calltree(tree,case_dir)

soiltemperature
|---->soilthermprop
|---->computegroundheatfluxandderiv
|---->computeheatdifffluxandfactor
|---->solvetemperature
|---->|---->setrhsvec
|---->|---->|---->setrhsvec_snow
|---->|---->|---->|---->setrhsvec_snowurban
|---->|---->|---->|---->|---->setrhsvec_snowurbannonroad
|---->|---->|---->|---->|---->setrhsvec_snowurbanroad
|---->|---->|---->|---->setrhsvec_snownonurban
|---->|---->|---->setrhsvec_standingsurfacewater
|---->|---->|---->setrhsvec_soil
|---->|---->|---->|---->setrhsvec_soilurban
|---->|---->|---->|---->|---->setrhsvec_soilurbannonroad
|---->|---->|---->|---->|---->setrhsvec_soilurbanroad
|---->|---->|---->|---->setrhsvec_soilnonurban
|---->|---->|---->|---->setrhsvec_soil_standingsurfacewater
|---->|---->setmatrix
|---->|---->|---->setmatrix_snow
|---->|---->|---->|---->setmatrix_snowurban
|---->|---->|---->|---->|---->setmatrix_snowurbannonroad
|---->|---->|---->|---->|---->setmatrix_snowurbanroad
|---->|---->|---->|---->setmatrix_snownonurban
|---->|---->

In [16]:
subname = 'laketemperature' 
sub:Subroutine = main_sub_dict[subname]
for child_sub in sub.child_Subroutine.values():
    print(child_sub.name)
    for subcall in child_sub.subroutine_call:
        print(subcall.subname)
        print(subcall.args)

soilthermprop_lake
laketemperature
[bounds => bounds, num_lakec => num_lakec, filter_lakec => filter_lakec, cv => tk, tk => cv, tktopsoillay => tktopsoillay, soilstate_vars => soilstate_vars]
tridiagonal_sr
laketemperature
[bounds => bounds, jtop => jtop, numf => num_lakec, filter => filter_lakec, a => a, b => b, c => c1, r => r, u => tx]
phasechange_lake
laketemperature
[bounds => bounds, num_lakec => num_lakec, filter_lakec => filter_lakec, cv => cv, cv_lake => cv_lake, lhabs => lhabs, energyflux_vars => energyflux_vars, lakestate_vars => lakestate_vars, dtime => dtime]
